# Development Notebook

### Imports

In [1]:
# -- public imports

from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch

In [2]:
# -- private import
from argminer.data import KaggleDataset

Import of ArgMiner successful


In [3]:
# -- dev imports

%load_ext autoreload
%autoreload 2
from python_files.data import create_labels_doc_level #, KaggleDataset
from python_files.run import train_longformer

IndentationError: expected an indented block (data.py, line 115)

## Model Configuration

### Tokenizer

In [4]:
model = 'google/bigbird-roberta-large'

tokenizer = BigBirdTokenizer.from_pretrained(model)
tokenizer = AutoTokenizer.from_pretrained(model, add_prefix_space=True)
# def need an encoding function to categoize the inputs or at least to release them, maybe need to store them together tbh?

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [5]:
tokenizer('humans humans').word_ids()

[None, 0, 1, None]

### Model and Optmizer

In [10]:
config_model = AutoConfig.from_pretrained(model) 
config_model.num_labels = 3 # might have to change if we're using the numbered discourse type as well
#config_model.save_pretrained('model')

model = AutoModelForTokenClassification.from_pretrained(model, config=config_model)
optimizer = torch.optim.Adam(params=model.parameters())

Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing BigBirdForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were n

## Data Preparation

In [154]:
texts = [
            'Hi my name is Yousef Nami',
            'Hi I am Shirin'
        ]
labels = [
    ['O', 'O', 'O', 'O', 'B-PERS', 'I-PERS'],
    ['O', 'O', 'O', 'B-PERS']
]

# TODO see if bert can accept text inputs?
labels_numeric = [
    [0, 0, 0, 0, 1, 2],
    [0, 0, 0, 1]
]

In [155]:
df = pd.DataFrame({
        'text': texts,
        'labels': labels_numeric
    })

In [156]:
dataset = KaggleDataset(
    df,
    tokenizer,
    max_length=20
)

In [159]:
for (inputs, targets) in dataset:
    print(inputs)
    

{'input_ids': tensor([   65, 16003,   717,  1539,   419,   676,   617,   992,   500,  6378,
           66,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'doc_ids': 0, 'word_ids': [0, 1, 2, 3, 4, 4, 4, 5, 5]}
{'input_ids': tensor([   65, 16003,   415,   817,  1012, 47489,    66,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'doc_ids': 1, 'word_ids': [0, 1, 2, 3, 3]}


In [ ]:
"You sef Na mi" -> "BNAME, BNAME, INAME, INAME"

## Data Training

In [11]:
train_loader = DataLoader(dataset, shuffle=True, batch_size=2)

NameError: name 'dataset' is not defined

In [ ]:
train_longformer(model=model, optimizer=optimizer , epochs=1, train_loader=train_loader)

## Test on Kaggle dataset

In [12]:
PATH = '../../data/kaggle/feedback-prize-2021/train'
FILE_PATH = '../../data/kaggle/feedback-prize-2021/train.csv'

df_texts = create_labels_doc_level(PATH, FILE_PATH)
#df_texts[['text', 'labels']].to_csv('../../../data/kaggle/df_cleaned.csv')

#df_kaggle = pd.read_csv('../../data/kaggle/df_cleaned.csv')
#df_kaggle.head()
df_texts.head()

,id,text,text_split,labels,range,labels_temp
0,3321A3E87AD3,I do agree that some students would benefit fr...,"[I, do, agree, that, some, students, would, be...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,DFEAEC512BAB,Should students design a summer project for sc...,"[Should, students, design, a, summer, project,...","[0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, ...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[3, 4, 4, 4, 4, 4, 4, 7, 8, 8, 8, 8, 8, 8, 8, ..."
2,2E4AFCD3987F,"Dear State Senator\n\n,\n\nIn the ruels of vot...","[Dear, State, Senator, ,, In, the, ruels, of, ...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 8, ...","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 8, 8, 8, 8, 8, ..."
3,EB6C2AF20BFE,People sometimes have a different opinion than...,"[People, sometimes, have, a, different, opinio...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,A91A08E523D5,"Dear senator,\n\nAs you know the Electoral Col...","[Dear, senator,, As, you, know, the, Electoral...","[0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [13]:
df_kaggle = df_texts[['text', 'labels']]
df_kaggle.head()

,text,labels
0,I do agree that some students would benefit fr...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,Should students design a summer project for sc...,"[0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, ..."
2,"Dear State Senator\n\n,\n\nIn the ruels of vot...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 8, ..."
3,People sometimes have a different opinion than...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,"Dear senator,\n\nAs you know the Electoral Col...","[0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [151]:
df_test =pd.DataFrame(data={'text':['hi my name is Yousef'], 'labels': [1]})

kaggle_dataset = KaggleDataset(df_test, tokenizer, max_length=20)

In [152]:
for (inputs, targets) in kaggle_dataset:
    print(inputs)
    print(targets)
    break

In [153]:
k_train_loader = DataLoader(kaggle_dataset, batch_size=2)
for (inputs, targets) in k_train_loader:
    print(inputs)
    break

IndexError: too many indices for tensor of dimension 0

In [31]:
train_longformer(model=model, optimizer=optimizer, epochs=1, train_loader=k_train_loader)

Attention type 'block_sparse' is not possible if sequence_length: 20 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


EPOCH 1 STARTED
---------------
tensor(1.0604, grad_fn=<NllLossBackward0>)
tensor([[[-0.1543, -0.0606, -0.1969],
         [-0.0912,  0.0348, -0.0372],
         [ 0.0957,  0.2102,  0.1286],
         [ 0.4059,  0.4303,  0.0561],
         [ 0.8208,  0.8731, -0.2832],
         [ 0.0943,  0.1394, -0.4872],
         [ 0.2177,  0.0998, -0.0414],
         [ 0.2388,  0.5826, -0.4362],
         [ 0.2919,  0.1661,  0.0572],
         [-0.2495,  0.1740, -0.3572],
         [ 0.3496,  0.3546,  0.2610],
         [ 0.2826,  0.4353, -0.3105],
         [-0.1331, -0.0448, -0.2625],
         [ 0.3561,  0.1659, -0.1571],
         [-0.1036, -0.0262, -0.2957],
         [ 0.0132,  0.1542, -0.4864],
         [ 0.2793,  0.2334, -0.1412],
         [-0.1528, -0.0184, -0.2182],
         [-0.1823, -0.0512, -0.2283],
         [-0.1329, -0.0350, -0.2510]]], grad_fn=<AddBackward0>) torch.Size([1, 20, 3])
Batch 1 complete. Time taken: load(0.0131), train(18.6), total(18.6). 


Exception: 

In [62]:
def active_logits(raw_logits, word_ids):
    word_ids = word_ids.view(-1)
    active_mask = word_ids.unsqueeze(1).expand(word_ids.shape[0], 3)
    print(active_mask)
    active_mask = active_mask != -1
    active_logits = raw_logits.view(-1, 3)
    print(active_logits)
    active_logits = torch.masked_select(active_logits, active_mask) # return 1dTensor
    active_logits = active_logits.view(-1, 3) 
    return active_logits

In [66]:
word_ids = tokenizer('Hi my name is سیس Yousef').word_ids()
L = len(word_ids)
word_ids = [word_id if word_id is not None else -1 for word_id in word_ids]
word_ids = torch.tensor(word_ids)

raw_logits = torch.rand((L, 3))
active_logits(raw_logits, word_ids)

tensor([[-1, -1, -1],
        [ 0,  0,  0],
        [ 1,  1,  1],
        [ 2,  2,  2],
        [ 3,  3,  3],
        [ 4,  4,  4],
        [ 4,  4,  4],
        [ 5,  5,  5],
        [ 5,  5,  5],
        [ 5,  5,  5],
        [-1, -1, -1]])
tensor([[0.9573, 0.3607, 0.6314],
        [0.4865, 0.2565, 0.6984],
        [0.4938, 0.6211, 0.6231],
        [0.1811, 0.8638, 0.6316],
        [0.6443, 0.3207, 0.4406],
        [0.1913, 0.6350, 0.8052],
        [0.2440, 0.1049, 0.7696],
        [0.5300, 0.6490, 0.8611],
        [0.2157, 0.6968, 0.4399],
        [0.8934, 0.0529, 0.4582],
        [0.0897, 0.0093, 0.6105]])


tensor([[0.4865, 0.2565, 0.6984],
        [0.4938, 0.6211, 0.6231],
        [0.1811, 0.8638, 0.6316],
        [0.6443, 0.3207, 0.4406],
        [0.1913, 0.6350, 0.8052],
        [0.2440, 0.1049, 0.7696],
        [0.5300, 0.6490, 0.8611],
        [0.2157, 0.6968, 0.4399],
        [0.8934, 0.0529, 0.4582]])

In [80]:
from torch.utils.data import Dataset
# need help with this
IGNORE_INDEX = -100
NON_LABEL = -1
LABELS_TO_IDS = {
    'O': 0,
    'B-PERS': 1,
    'I-PERS': 2
}

df = pd.DataFrame({
        'text': texts,
        'entities': labels
    })

SUBTOKENS = True
class FeedbackPrizeDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, has_labels):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.has_labels = has_labels
    
    def __getitem__(self, index):
        text = self.data.text[index]
        encoding = self.tokenizer(
            text.split(),
            is_split_into_words = True,
            padding = 'max_length',
            truncation = True,
            max_length = self.max_len
        )
        word_ids = encoding.word_ids()

        # targets
        if self.has_labels:
            word_labels = self.data.entities[index]
            prev_word_idx = None
            labels_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    labels_ids.append(IGNORE_INDEX)
                elif word_idx != prev_word_idx:
                    labels_ids.append(LABELS_TO_IDS[word_labels[word_idx]])
                else:
                    if SUBTOKENS:
                        labels_ids.append(LABELS_TO_IDS[word_labels[word_idx]])
                    else:
                        labels_ids.append(IGNORE_INDEX)
                prev_word_idx = word_idx
            encoding['labels'] = labels_ids
        # convert to torch.tensor
        item = {k: torch.as_tensor(v) for k, v in encoding.items()}
        word_ids2 = [w if w is not None else NON_LABEL for w in word_ids]
        item['word_ids'] = torch.as_tensor(word_ids2)
        return item

    def __len__(self):
        return self.len
    
dset = FeedbackPrizeDataset(df.rename(columns={'labels':'entities'}),  tokenizer,  10,True)

In [82]:
def active_labels(labels):
    active_mask = labels.view(-1) != IGNORE_INDEX
    active_labels = torch.masked_select(labels.view(-1), active_mask)
    return active_labels

In [83]:
for item in dset:
    input_ids, attention_mask, labels, word_ids = item.values()
    print(labels)
    print(active_labels(labels))
    break

tensor([-100,    0,    0,    0,    0,    1,    1,    1,    2, -100])
tensor([0, 0, 0, 0, 1, 1, 1, 2])


# Model evalation

In [28]:
from python_files.run import evaluate

t1 = torch.tensor([[1,2,3,4], [5,6,7,8]])
t2 = torch.tensor([[1,3,2,4], [6, 7, 5, 8]])

print(t1==t2)
(t1 == t2).sum(dim=1)

tensor([[ True, False, False,  True],
        [False, False, False,  True]])


tensor([2, 1])

In [31]:
(t1 == 1).nonzero()

tensor([[0, 0]])

In [45]:
df_test = pd.DataFrame(
    {'t1': [set([1,2]), set([4,5])],
    't2':[set([1,1]), set([4,5])]}
)

In [49]:
df_test['gt_and_pred'] = df_test.apply(lambda x: x.t1.intersection(x.t2), axis=1)

In [50]:
df_test['gt_and_pred']

0       {1}
1    {4, 5}
Name: gt_and_pred, dtype: object

In [55]:
torch.tensor([[1,2,3,4], [1,1,1,2]]).apply_(lambda x: x*2)

tensor([[2, 4, 6, 8],
        [2, 2, 2, 4]])

TypeError: torch.LongTensor is not a Module subclass

In [72]:
torch.tensor_split(torch.arange(7), [])

(tensor([0, 1, 2, 3, 4, 5, 6]),)

In [298]:
def _get_zero_start_ids(iterable, split_by=[0]):
    """ TODO run unit tests on this """

    split_by = set(split_by)
    add_item = float(iterable[0]) in split_by
    zero_start_ids = []
    for i, item in enumerate(iterable):
        if (float(item) in split_by) ^ add_item:
            add_item = not add_item
            zero_start_ids.append(i)
    return zero_start_ids

In [300]:
tensor = torch.tensor([0, 0, 1,2,3, 0, 3,5,0,0,0,5,6])
split_ids = _get_zero_start_ids(tensor)
print(split_ids)
torch.tensor_split(tensor, split_ids)


[2, 5, 6, 8, 11]


[autoreload of python_files.run failed: Traceback (most recent call last):
  File "/Users/yousefnami/opt/anaconda3/envs/argument-mining/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/yousefnami/opt/anaconda3/envs/argument-mining/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/yousefnami/opt/anaconda3/envs/argument-mining/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 839, in exec_module
  File "<frozen importlib._bootstrap_external>", line 976, in get_code
  File "<frozen importlib._bootstrap_external>", line 906, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/yousefnami/Desktop/Main/0.Educa

(tensor([0, 0]),
 tensor([1, 2, 3]),
 tensor([0]),
 tensor([3, 5]),
 tensor([0, 0, 0]),
 tensor([5, 6]))

In [217]:
torch.tensor_split(torch.tensor([0, 0, 
1,2,3,5,0,0,0,5,6]), [4, 7])

(tensor([0, 0, 1, 2]), tensor([3, 5, 0]), tensor([0, 0, 5, 6]))

In [111]:
tokenizer('Hi my name \nis Yousef')

CUDA device NOT detected. Using CPU...


{'input_ids': [65, 16003, 717, 1539, 321, 100, 372, 676, 617, 992, 66], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [125]:
test = (torch.rand((2,4,3)) * 10).int()
test.shape

torch.Size([2, 4, 3])

In [130]:
probas, values = test.max(dim=-1)

In [160]:
df = pd.concat(
    [pd.DataFrame(data={'id':i, 'probas':probas, 'values':values}) for i, (probas, values) in enumerate(zip(probas, values))]
)

CUDA device NOT detected. Using CPU...


In [161]:
df

,id,probas,values
0,0,6,0
1,0,5,1
2,0,9,1
3,0,6,0
0,1,8,0
1,1,7,1
2,1,9,2
3,1,9,1


In [164]:
df = pd.concat(
    [pd.DataFrame(data={'probas':probas, 'values':values, 'word_ids':[0,1,1,2]}).groupby('word_ids').head(1) for i, (probas, values) in enumerate(zip(probas, values))]
)
df

In [166]:
df['ids'] = [0,0,0,1,1,1]

CUDA device NOT detected. Using CPU...


In [193]:
g = df.groupby('ids').agg({col: lambda x: x.values.flatten().tolist() for col in df.columns}).drop('ids', axis=1)
g

,probas,values,word_ids
ids,,,
0,"[6, 5, 6]","[0, 1, 0]","[0, 1, 2]"
1,"[8, 7, 9]","[0, 1, 1]","[0, 1, 2]"


In [5]:
import base64
def encode_model_name(model_name, epoch):
    model_name_b = model_name.encode('ascii')
    encoded_model_name = base64.b64encode(model_name_b).decode('ascii')
    return encoded_model_name + f'_{epoch}'

In [6]:
encode_model_name('google/bigbird', 1)

'Z29vZ2xlL2JpZ2JpcmQ=_1'